In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\gkart\\Desktop\\1-ProjectENDtoEND\\chicken_disease_classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\gkart\\Desktop\\1-ProjectENDtoEND\\chicken_disease_classification'

In [5]:
# Step -Creating/Updating entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class PrepareBaseModelConfig:
    root_dir :Path
    base_model_path : Path
    updated_base_model_path : Path
    params_image_size :list
    params_learning_rate :float
    params_include_top : bool
    params_weights: str 
    params_classes : int

In [6]:
from src.chickenDiseaseClassifier.constants import *
from src.chickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [ ]:
# Step - Creating/Updating Configuration Manager

class ConfigurationManager:
    def __init__(self,config = CONFIG_FILE_PATH, param = PARAM_FILE_PATH):
        self.config = read_yaml(config)
        self.param = read_yaml(param)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_configuration(self) -> PrepareBaseModelConfig :
        config = self.config.prepare_base_model

        prepare_base_model_config = PrepareBaseModelConfig(
                                                root_dir = Path(config.root_dir),
                                                base_model_path = Path(config.base_model_path),
                                                updated_base_model_path= Path(config.updated_base_model_path),
                                                params_image_size = self.param.IMAGE_SIZE,
                                                params_learning_rate = self.param.LEARNING_RATE,
                                                params_include_top = self.param.INCLUDE_TOP,
                                                params_weights = self.param.WEIGHTS,
                                                params_classes = self.param.CLASSES
                                                )
        return prepare_base_model_config

In [50]:
# Step - Creating/Updating Components

import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten,Dense
from keras.models import Sequential

class PrepareBaseModel:

    def __init__(self, config :PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = VGG16(include_top = self.config.params_include_top, input_shape = self.config.params_image_size, weights = self.config.params_weights)
        self.save_model(path = self.config.base_model_path, model = self.model)
    
    @staticmethod
    def save_model(path,model):
        model.save(path)

    @staticmethod
    def prepare_full_base_model(model,learning_rate,classes):

        for layer in model.layers:
            layer.trainable = False
        
        model = Sequential(model)
        model.add(Flatten())
        dense_layer1 = Dense(units = classes, activation = "softmax")
        model.add(dense_layer1)

        model.compile( optimizer= tf.keras.optimizers.SGD(learning_rate=learning_rate),loss = tf.keras.losses.CategoricalCrossentropy(),metrics = ['accuracy'])
        model.summary()
        return model 
    
    def update_full_model(self):
        self.full_model = self.prepare_full_base_model(model = self.model,learning_rate = self.config.params_learning_rate, classes = self.config.params_classes)
        self.save_model(self.config.updated_base_model_path,self.full_model)
   

In [51]:
# updating/creating the pipeline

try:
    configuration_manager = ConfigurationManager()
    prepare_base_model_config = configuration_manager.get_prepare_base_model_configuration()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    print(prepare_base_model_config.base_model_path)
    prepare_base_model.get_base_model()
    prepare_base_model.update_full_model()
except Exception as e:
    raise e

[2023-08-01 15:26:51,866: INFO: common: yaml file : config\config.yaml loaded sucesfully]
[2023-08-01 15:26:52,159: INFO: common: yaml file : params.yaml loaded sucesfully]
[2023-08-01 15:26:52,354: INFO: common: Create directory at : artifacts]
artifacts\prepare_base_model\base_model.h5
[2023-08-01 15:27:26,796: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]


c:\Users\gkart\Anaconda3\envs\chicken\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 14764866 (56.32 MB)
Trainable params: 50178 (196.01 KB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


NameError: name 'save_model' is not defined